# Tutorial: DataFrames with RLM

DSPy provides first-class support for pandas DataFrames in the RLM (Recursive Language Model) module. DataFrames are automatically serialized via Parquet format, preserving dtypes, and rich metadata is provided to the LLM.

> **Important**: `dspy.DataFrame` should only be used with `dspy.RLM`. Other modules like `ChainOfThought` or `Predict` will only see a string representation of the DataFrame, not the actual data. RLM provides a Python sandbox where the DataFrame is available for code execution.

Install the latest DSPy via `pip install -U dspy` and follow along.

## 1) Setup

First, let's configure DSPy with an LM and create a sample DataFrame.

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Pydantic serializer warnings")

import dspy
import pandas as pd

# Configure your LM
lm = dspy.LM("anthropic/claude-sonnet-4-5-20250929", max_tokens=16000)
dspy.configure(lm=lm)

In [ ]:
# Create a sample DataFrame
dataframe = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "city": ["New York", "Los Angeles", "Chicago", "Houston", "Miami"]
})

dataframe

## 2) Using `dspy.DataFrame` in Signatures

To use a DataFrame as an input field in a DSPy Signature, use the `dspy.DataFrame` type annotation. This tells Pydantic how to handle the DataFrame type.

In [ ]:
class DocWriter(dspy.Signature):
    """Write documentation for the provided data."""
    
    dataframe: dspy.DataFrame = dspy.InputField()
    documentation: str = dspy.OutputField(desc="Generated markdown documentation.")

## 3) Running RLM with DataFrames

Now we can use `dspy.RLM` to process the DataFrame. The RLM module will:

1. Serialize the DataFrame to Parquet format (preserving dtypes)
2. Provide rich metadata to the LLM (shape, columns, dtypes, sample rows)
3. Make the DataFrame available in the Python sandbox for code execution

In [ ]:
doc_writer = dspy.RLM(
    DocWriter,
    max_iterations=10,
    verbose=True
)

result = doc_writer(dataframe=dataframe)

In [ ]:
print(result.documentation)

## 4) How It Works

When you pass a DataFrame to RLM:

1. **Serialization**: The DataFrame is serialized to Parquet format using PyArrow, which preserves data types (int, float, datetime, categorical, etc.)

2. **Metadata**: The LLM receives rich metadata about the DataFrame:
   - Shape (rows x columns)
   - Column names and dtypes
   - Null value counts
   - Sample rows (first and last 3 rows)

3. **Sandbox Access**: The DataFrame is made available in the Python sandbox, where the LLM-generated code can access it directly using pandas operations.

## 5) Advanced Example: Data Analysis

Let's try a more complex example where the LLM analyzes the data from the UCI Adult / Census Income dataset.

In [ ]:
import pandas as pd

input_data = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None
)

In [ ]:
from typing import Optional, List

class DataAnalyst(dspy.Signature):
    """Analyze the data and provide insights. Do just a very basic analysis, nothing substantial. When generating images, do this using in-memory IO operations and return them as base64-encoded strings."""
    
    input_data: dspy.DataFrame = dspy.InputField()
    analysis_title: str = dspy.OutputField()
    analysis_summary: str = dspy.OutputField(desc="Detailed analysis with statistics and insights.")
    analysis_graphs: List[dspy.Image] = dspy.OutputField(desc="A series of graphs or images related to the analysis. These should be encoded as base64 urls.")

analyst = dspy.RLM(DataAnalyst, max_iterations=10, verbose=True)
result = analyst(input_data=input_data)

In [ ]:
print(result)

In [ ]:
from IPython.display import Image, display

display(Image(url=result.analysis_graphs[0].url))

## Summary

- Use `dspy.DataFrame` as the type annotation for DataFrame input fields
- DataFrames are serialized via Parquet, preserving all dtypes
- Rich metadata (shape, columns, dtypes, samples) is provided to the LLM
- The DataFrame is available in the RLM sandbox for pandas operations
- Works seamlessly with other input types (strings, numbers, etc.)